In [16]:
import os
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import mmseg_custom
import mmcv_custom

import pandas as pd
import numpy as np
import json
import cv2

import albumentations as A

In [17]:
# Need to set
CONFIG_PATH = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/segmentation/configs/_custom_/mask2former_beitv2_adapter_large_896_80k_cocostuff164k_ss.py'
TEST_IMAGES_PATH = '/opt/ml/input/data/test_images'
ITER = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/tools/beitv2_adapter_best_mIoU_iter_80000'
WORK_DIR = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/mmsegmentation/work_dirs/upernet_swin_large_patch4_window7_512x512_pretrain_224x224_22K_160k_ade20k_0/'

In [18]:
cfg = Config.fromfile(CONFIG_PATH)
root=TEST_IMAGES_PATH

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.work_dir = WORK_DIR
# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
# checkpoint_path = os.path.join(cfg.work_dir, f'{ITER}.pth')
checkpoint_path = f'{ITER}.pth'

In [19]:
dataset = build_dataset(cfg.data.test)
if len(dataset) != 819:
        raise AssertionError('Test dataset should 819 image. Check your test.json')
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2023-01-01 14:06:23,753 - mmseg - INFO - Loaded 819 images


Position interpolate for blocks.0.attn.relative_position_bias_table from 27x27 to 111x111
x = [-54.999898443794464, -44.33237151632927, -35.57463714019768, -28.384787803571555, -22.482127222283516, -17.63621177863096, -13.657853856192851, -10.39173583513709, -7.71034639370451, -5.509002385228651, -3.701761941608667, -2.2180692481994626, -1, 0, 1, 2.2180692481994626, 3.701761941608667, 5.509002385228651, 7.71034639370451, 10.39173583513709, 13.657853856192851, 17.63621177863096, 22.482127222283516, 28.384787803571555, 35.57463714019768, 44.33237151632927, 54.999898443794464]
dx = [-55. -54. -53. -52. -51. -50. -49. -48. -47. -46. -45. -44. -43. -42.
 -41. -40. -39. -38. -37. -36. -35. -34. -33. -32. -31. -30. -29. -28.
 -27. -26. -25. -24. -23. -22. -21. -20. -19. -18. -17. -16. -15. -14.
 -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.   0.
   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26

2023-01-01 14:06:34,595 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc_norm.weight, fc_norm.bias, head.weight, head.bias

missing keys in source state_dict: blocks.0.attn.relative_position_index, blocks.1.attn.relative_position_index, blocks.2.attn.relative_position_index, blocks.3.attn.relative_position_index, blocks.4.attn.relative_position_index, blocks.5.attn.relative_position_index, blocks.6.attn.relative_position_index, blocks.7.attn.relative_position_index, blocks.8.attn.relative_position_index, blocks.9.attn.relative_position_index, blocks.10.attn.relative_position_index, blocks.11.attn.relative_position_index, blocks.12.attn.relative_position_index, blocks.13.attn.relative_position_index, blocks.14.attn.relative_position_index, blocks.15.attn.relative_position_index, blocks.16.attn.relative_position_index, blocks.17.attn.relative_position_index, blocks.18.attn.relative_position_index, blocks.19.attn.relative_po

load checkpoint from local path: /opt/ml/input/level2_semanticsegmentation_cv-level2-cv-16/tools/beitv2_adapter_best_mIoU_iter_80000.pth


In [20]:
output = single_gpu_test(model, data_loader)

[                                                  ] 0/819, elapsed: 0s, ETA:

RuntimeError: CUDA out of memory. Tried to allocate 602.00 MiB (GPU 0; 31.75 GiB total capacity; 4.71 GiB already allocated; 584.50 MiB free; 5.34 GiB reserved in total by PyTorch)

In [ ]:
# sample_submisson.csv 열기
submission = pd.read_csv('sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# set resize
input_size = 512
output_size = 256
transformed = A.Compose([A.Resize(output_size, output_size)])
		

# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    temp_mask = []
    mask = np.array(predict, dtype='uint8')
    mask = transformed(image=mask)
    temp_mask.append(mask['image'])
    oms = np.array(temp_mask)
    cv2.imwrite(os.path.join('../tools/mask_test/', file_name), oms)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(f'submission_{ITER}.csv', index=False)

print('Done')

Done
